<a href="https://colab.research.google.com/github/walkerjian/DailyCode/blob/main/FilesystemTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Suppose we represent our file system by a string in the following manner:

The string "dir\n\tsubdir1\n\tsubdir2\n\t\tfile.ext" represents:

dir
    subdir1
    subdir2
        file.ext
The directory dir contains an empty sub-directory subdir1 and a sub-directory subdir2 containing a file file.ext.

The string "dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext" represents:

dir
    subdir1
        file1.ext
        subsubdir1
    subdir2
        subsubdir2
            file2.ext
The directory dir contains two sub-directories subdir1 and subdir2. subdir1 contains a file file1.ext and an empty second-level sub-directory subsubdir1. subdir2 contains a second-level sub-directory subsubdir2 containing a file file2.ext.

We are interested in finding the longest (number of characters) absolute path to a file within our file system. For example, in the second example above, the longest absolute path is "dir/subdir2/subsubdir2/file2.ext", and its length is 32 (not including the double quotes).

Given a string representing the file system in the above format, return the length of the longest absolute path to a file in the abstracted file system. If there is no file in the system, return 0.

Note:

The name of a file contains at least a period and an extension.

The name of a directory or sub-directory will not contain a period.

requirements:
````
1) use the MVC paradigm.
2) extensively document your code with a docstring for the initial problem as specified.
3) all code to be uninterrupted and not truncated.
4) extensively test the code; write a test function to test the code with at least 10 test examples. Make sure the test harness does not interrupt the output of the test cases, which should include the sample cases given to you. All output needs to include the original example or test case, and complete output of the solution.
5) do not make up solutions, make sure your solution is correct & adheres to all requirements above.
6) use PEP8 & nice formatting rules for word wrap etc.
````
use a hybrid tree structure - a tree node contains links to sub directories and also a list of filenames in the directory, when you are determining the length of the path construct each path from the tree (in the format: dir/subdir2/subsubdir2/file2.ext) and keep a count of the length of each one, when you have traversed the full tree and generated all the paths and calculated the longest one, return the longest path as a string, its lengths and also any tied paths too

In [2]:
# TreeNode class to represent directories and files.
class TreeNode:
    def __init__(self, name: str, level: int):
        self.name = name
        self.level = level
        self.subdirs = []  # List of sub-directory tree nodes.
        self.files = []    # List of filenames in the current directory.

    def add_subdir(self, subdir):
        self.subdirs.append(subdir)

    def add_file(self, filename):
        self.files.append(filename)

    def __str__(self):
        return self.name

# Model class
class FilesystemTree:
    def __init__(self):
        self.root = TreeNode("root", -1)

    def build_tree(self, filesystem: str):
        items = filesystem.split('\n')
        stack = [self.root]
        for item in items:
            level = item.count('\t')
            item = item.strip('\t')
            if '.' in item:
                stack[level].add_file(item)
            else:
                node = TreeNode(item, level)
                stack[level].add_subdir(node)
                stack = stack[:level+1] + [node]

    def construct_paths(self, node: TreeNode = None, current_path: str = "") -> list:
        if node is None:
            node = self.root
        if node.name != "root":
            current_path += node.name + "/"
        paths = []
        for file in node.files:
            paths.append(current_path + file)
        for subdir in node.subdirs:
            paths.extend(self.construct_paths(subdir, current_path))
        return paths

# View class
class FilesystemView:
    @staticmethod
    def display_output(longest_path, longest_length, tied_paths):
        print(f"Longest Path: {longest_path}")
        print(f"Length: {longest_length}")
        print(f"Tied Paths: {tied_paths}")

# Controller class
class FilesystemController:
    def __init__(self):
        self.model = FilesystemTree()
        self.view = FilesystemView()

    def get_longest_path(self, filesystem: str):
        self.model = FilesystemTree()  # Reset the model tree
        self.model.build_tree(filesystem)
        paths = self.model.construct_paths()
        longest_path, longest_length, tied_paths = find_longest_paths(paths)
        self.view.display_output(longest_path, longest_length, tied_paths)

# Helper function to find the longest paths
def find_longest_paths(paths: list) -> tuple:
    if not paths:
        return ("", 0, [])
    paths.sort(key=len, reverse=True)
    longest_path = paths[0]
    longest_length = len(longest_path)
    tied_paths = [path for path in paths if len(path) == longest_length]
    return (longest_path, longest_length, tied_paths)

# Test harness
def test():
    test_cases = [
        ("dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext",
         ('dir/subdir2/subsubdir2/file2.ext', 32, ['dir/subdir2/subsubdir2/file2.ext'])),
        ("dir\n\tsubdir1\n\tsubdir2\n\t\tfile.ext",
         ('dir/subdir2/file.ext', 18, ['dir/subdir2/file.ext'])),
        ("dir\n\tsubdir1\n\tsubdir2",
         ('', 0, [])),
        ("dir\n\tsubdir1\n\t\tfile.ext",
         ('dir/subdir1/file.ext', 20, ['dir/subdir1/file.ext'])),
        ("dir\n\tsubdir1\n\t\tsubsubdir\n\t\t\tfile.ext",
         ('dir/subdir1/subsubdir/file.ext', 30, ['dir/subdir1/subsubdir/file.ext'])),
        ("dir\n\tsubdir1\n\t\tfile1.ext\n\tsubdir2\n\t\tfile2.ext",
         ('dir/subdir1/file1.ext', 20, ['dir/subdir1/file1.ext', 'dir/subdir2/file2.ext'])),
        ("dir\n\tsubdir1\n\t\tsubsubdir\n\t\t\tfile.ext\n\tsubdir2\n\t\tfile.ext",
         ('dir/subdir1/subsubdir/file.ext', 30, ['dir/subdir1/subsubdir/file.ext'])),
        ("dir\n\tfile.ext",
         ('dir/file.ext', 12, ['dir/file.ext'])),
        ("dir\n\tsubdir\n\t\tfile1.ext\n\t\tfile2.ext",
         ('dir/subdir/file1.ext', 21, ['dir/subdir/file1.ext', 'dir/subdir/file2.ext'])),
        ("dir\n\tsubdir\n\t\tsubsubdir\n\t\t\tsubsubsubdir\n\t\t\t\tfile.ext",
         ('dir/subdir/subsubdir/subsubsubdir/file.ext', 41, ['dir/subdir/subsubdir/subsubsubdir/file.ext']))
    ]

    controller = FilesystemController()
    for i, (input_str, expected_output) in enumerate(test_cases, 1):
        print(f"Test Case {i}")
        print("-" * 20)
        print(f"Input:\n{input_str}\n")
        controller.get_longest_path(input_str)
        print(f"Expected Output: {expected_output}\n\n")

# Running the test harness
test()


Test Case 1
--------------------
Input:
dir
	subdir1
		file1.ext
		subsubdir1
	subdir2
		subsubdir2
			file2.ext

Longest Path: dir/subdir2/subsubdir2/file2.ext
Length: 32
Tied Paths: ['dir/subdir2/subsubdir2/file2.ext']
Expected Output: ('dir/subdir2/subsubdir2/file2.ext', 32, ['dir/subdir2/subsubdir2/file2.ext'])


Test Case 2
--------------------
Input:
dir
	subdir1
	subdir2
		file.ext

Longest Path: dir/subdir2/file.ext
Length: 20
Tied Paths: ['dir/subdir2/file.ext']
Expected Output: ('dir/subdir2/file.ext', 18, ['dir/subdir2/file.ext'])


Test Case 3
--------------------
Input:
dir
	subdir1
	subdir2

Longest Path: 
Length: 0
Tied Paths: []
Expected Output: ('', 0, [])


Test Case 4
--------------------
Input:
dir
	subdir1
		file.ext

Longest Path: dir/subdir1/file.ext
Length: 20
Tied Paths: ['dir/subdir1/file.ext']
Expected Output: ('dir/subdir1/file.ext', 20, ['dir/subdir1/file.ext'])


Test Case 5
--------------------
Input:
dir
	subdir1
		subsubdir
			file.ext

Longest Path: di

Eifficient Solution

In [3]:
def efficient_longest_path(filesystem: str) -> tuple:
    """
    Efficiently determine the longest path and any paths that have the same length.

    :param filesystem: String representation of the file system.
    :return: Tuple containing the longest path, its length, and any tied paths.
    """
    path_lengths = {0: 0}  # Stores the cumulative path length at each depth level.
    current_paths = {0: ""}  # Stores the current path up to each depth level.
    max_length = 0
    max_paths = []

    for line in filesystem.split('\n'):
        depth = line.count('\t')  # Determine the depth level based on the number of tabs.
        name = line.strip('\t')   # Remove the tabs to get the name.

        # If it's a file
        if '.' in name:
            full_path = current_paths[depth] + name
            length = len(full_path)
            if length > max_length:
                max_length = length
                max_paths = [full_path]
            elif length == max_length:
                max_paths.append(full_path)
        else:
            # If it's a directory
            current_paths[depth + 1] = current_paths[depth] + name + "/"
            # Update the path length storage for the current depth level.
            path_lengths[depth + 1] = path_lengths[depth] + len(name) + 1

    # If there's no file in the filesystem
    if max_length == 0:
        return ("", 0, [])

    return (max_paths[0], max_length, max_paths)

# Running the test harness for the efficient solution again
# Testing the efficient solution using the comprehensive test harness

def test_efficient():
    """
    Test function to extensively test the efficient solution.
    """
    test_cases = [
        # Sample test cases
        ("dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext",
         ('dir/subdir2/subsubdir2/file2.ext', 32, ['dir/subdir2/subsubdir2/file2.ext'])),

        ("dir\n\tsubdir1\n\tsubdir2\n\t\tfile.ext",
         ('dir/subdir2/file.ext', 18, ['dir/subdir2/file.ext'])),

        # Additional test cases
        ("dir\n\tsubdir1\n\tsubdir2",
         ('', 0, [])),

        ("dir\n\tsubdir1\n\t\tfile.ext",
         ('dir/subdir1/file.ext', 20, ['dir/subdir1/file.ext'])),

        ("dir\n\tsubdir1\n\t\tsubsubdir\n\t\t\tfile.ext",
         ('dir/subdir1/subsubdir/file.ext', 30, ['dir/subdir1/subsubdir/file.ext'])),

        ("dir\n\tsubdir1\n\t\tfile1.ext\n\tsubdir2\n\t\tfile2.ext",
         ('dir/subdir1/file1.ext', 20, ['dir/subdir1/file1.ext', 'dir/subdir2/file2.ext'])),

        ("dir\n\tsubdir1\n\t\tsubsubdir\n\t\t\tfile.ext\n\tsubdir2\n\t\tfile.ext",
         ('dir/subdir1/subsubdir/file.ext', 30, ['dir/subdir1/subsubdir/file.ext'])),

        ("dir\n\tfile.ext",
         ('dir/file.ext', 12, ['dir/file.ext'])),

        ("dir\n\tsubdir\n\t\tfile1.ext\n\t\tfile2.ext",
         ('dir/subdir/file1.ext', 21, ['dir/subdir/file1.ext', 'dir/subdir/file2.ext'])),

        ("dir\n\tsubdir\n\t\tsubsubdir\n\t\t\tsubsubsubdir\n\t\t\t\tfile.ext",
         ('dir/subdir/subsubdir/subsubsubdir/file.ext', 41, ['dir/subdir/subsubdir/subsubsubdir/file.ext']))
    ]

    for i, (input_str, expected_output) in enumerate(test_cases, 1):
        print(f"Test Case {i}")
        print("-" * 20)
        print(f"Input:\n{input_str}\n")
        result = efficient_longest_path(input_str)
        print(f"Actual Output: {result}")
        print(f"Expected Output: {expected_output}\n\n")

# Running the test harness for the efficient solution
test_efficient()


Test Case 1
--------------------
Input:
dir
	subdir1
		file1.ext
		subsubdir1
	subdir2
		subsubdir2
			file2.ext

Actual Output: ('dir/subdir2/subsubdir2/file2.ext', 32, ['dir/subdir2/subsubdir2/file2.ext'])
Expected Output: ('dir/subdir2/subsubdir2/file2.ext', 32, ['dir/subdir2/subsubdir2/file2.ext'])


Test Case 2
--------------------
Input:
dir
	subdir1
	subdir2
		file.ext

Actual Output: ('dir/subdir2/file.ext', 20, ['dir/subdir2/file.ext'])
Expected Output: ('dir/subdir2/file.ext', 18, ['dir/subdir2/file.ext'])


Test Case 3
--------------------
Input:
dir
	subdir1
	subdir2

Actual Output: ('', 0, [])
Expected Output: ('', 0, [])


Test Case 4
--------------------
Input:
dir
	subdir1
		file.ext

Actual Output: ('dir/subdir1/file.ext', 20, ['dir/subdir1/file.ext'])
Expected Output: ('dir/subdir1/file.ext', 20, ['dir/subdir1/file.ext'])


Test Case 5
--------------------
Input:
dir
	subdir1
		subsubdir
			file.ext

Actual Output: ('dir/subdir1/subsubdir/file.ext', 30, ['dir/subdir1/